## Парсинг товаров с сайта Перекресток. 


Парсинг интернет магазина Перекресток с помощью библиотеки BeautifulSoup

In [1]:
!pip install -q tqdm
!pip install -q ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


Импортируем нужные библиотеки

In [2]:
import pandas as pd 
import requests  # импорт библиотеки для запросов к серверу
from bs4 import (
    BeautifulSoup,
)  # импорт библиотеки для автоматического парсинга странички
import time
import os
from tqdm.notebook import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# настройки отображения

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', None)
pd.set_option("max_colwidth", 200)

In [3]:
# функции для фильтрации

def quantity_filter(row):
    if row and len(row)>0:
        a = row.split('₽/')
        if len(a)>1:
            return a[1]
    return ''

def price_filter(row):
    if row and len(row)>0:
        a = row.split('₽')
        if len(a)>=1:
            return a[0].strip().replace(' ', '').replace(',', '.')
    return np.nan

In [4]:
# количество объектов на страницу
items_per_page = 48

# номер категории начала (категория "Здоровье")
pages_start = 64 

# номер категори конца, не включая
pages_end = 65 

# переменные
all_products = []
all_price = []
all_exists = []

csv_file = 'products'+'_'+str(pages_start)+'.csv'
csv_file_exists = os.path.exists(csv_file)


# цикл для парсинга
if csv_file_exists:
    products = pd.read_csv(csv_file)

else:
    category = 'unknown' # Название категории
    
    for i in trange(pages_start, pages_end):
        j = 1
        pbar = tqdm()

        URL = 'https://www.perekrestok.ru/cat/c/'+str(i)
        while True:
            req = requests.get(URL)  # GET-запрос
            if req.status_code != 200:
                break

            soup = BeautifulSoup(req.text, 'lxml')
            category = soup.find('h1', attrs={'class' : 'page-header__title'}).text 
            
            name_products = []  # Список, в котором хранятся названия товаров
            for row in soup.find_all(
                'div', attrs={'class': 'product-card__title'}
            ):
                name_products.append(row.text)

            price = []  # Cписок, в котором хранятся цены на товар
            for row in soup.find_all(
                'div', attrs={'class': 'product-card__pricing'}
            ):
                price.append(row.text)

            name_exists = []
            for row in soup.find_all(
                'div', attrs={'class': 'product-card__balance-badge'} # Наличие
            ):
                name_exists.append(row.text)

            all_products.append(name_products)
            all_price.append(price)
            all_exists.append(name_exists)

            if len(name_products) < items_per_page:
                break
                
            if len(name_products) > items_per_page:
                break
            
            j += 1
            pbar.update(1)
            URL = 'https://www.perekrestok.ru/cat/c/'+str(i)+'/?page='+str(j)

            if j % 50 == 0: # Пауза 3 сек каждые 50 страниц
                time.sleep(3) 
        
    
    products_data = {} # создаем словарь

    for i in range(len(all_products)):
        for j in range(len(all_products[i])):
            products_data[str(i+1)+'.'+str(j+1)] = [all_products[i][j], all_price[i][j], all_exists[i][j], category]
    
    # преобразуем в датафрейм
    products = pd.DataFrame.from_dict(products_data, columns = ['name','price', 'exists', 'category'], orient = 'index')
    
    # выносим столбцы с единицей измерения и ценой
    products['quantity'] = products['price'].apply(lambda x: quantity_filter(x))
    products['price'] = products['price'].apply(lambda x: price_filter(x))
    
    # удаляем строки, в которых цена и единица измерения не указаны, удаляем дубликаты
    products = products.dropna(subset=['price', 'quantity']).drop_duplicates(ignore_index=True)
    
    #преобразуем в csv и сохраняем 
    products.to_csv(csv_file, index = False)

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [5]:
# посмотрим общую информацию

products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927 entries, 0 to 926
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      927 non-null    object
 1   price     927 non-null    object
 2   exists    927 non-null    object
 3   category  927 non-null    object
 4   quantity  927 non-null    object
dtypes: object(5)
memory usage: 36.3+ KB


In [6]:
# посмотрим первые 5 строк

products.head()

,name,price,exists,category,quantity
0,"Печенье Зелёная Линия Воздушные колечки заварное, 100г",49.99,В наличии много,Здоровье,шт
1,"Батончик глазированный Snaq Fabriq арахис карамель, 50г",69.99,В наличии много,Здоровье,шт
2,"Напиток овсяный Nemoloko шоколадный 3.2%, 1л",129.99,В наличии много,Здоровье,шт
3,"Лапша Imperial Cuisine гречневая соба без пучков, 400г",39.99,В наличии много,Здоровье,шт
4,"Мука Garnec рисовая без глютена, 500г",59.99,В наличии много,Здоровье,шт
